HDB Resale Transaction bot - by M. W. Ho (c) 2022


In [1]:
!pip install python-telegram-bot

import pandas as pd
import requests
import telegram

bot = telegram.Bot(token='5218309496:AAHV2gIFLgNSK37D98kvFScBiLMxorHotJU')
print("\n",bot.get_me(),"\n")

###
from telegram.ext import Updater
updater = Updater(token='5218309496:AAHV2gIFLgNSK37D98kvFScBiLMxorHotJU', use_context=True)
dispatcher = updater.dispatcher

###
import logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                     level=logging.INFO)


 {'username': 'HDBResaleTx_bot', 'supports_inline_queries': False, 'is_bot': True, 'can_read_all_group_messages': False, 'can_join_groups': True, 'first_name': 'HDBResaleTx', 'id': 5218309496} 



In [2]:
###
from telegram import Update
from telegram.ext import CallbackContext

def start(update: Update, context: CallbackContext):
    context.bot.send_message(chat_id=update.effective_chat.id, text="HDBRT Bot - Get latest HDB Resale Transactions.\nType / for commands.")

def last10(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,remaining_lease,storey_range&sort=month%20desc&limit=10&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','remaining_lease','storey_range','floor_area_sqm','flat_model','month'])
    context.bot.send_message(chat_id=update.effective_chat.id, text=str(df))

def last20(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,remaining_lease,storey_range&sort=month%20desc&limit=20&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','remaining_lease','storey_range','floor_area_sqm','flat_model','month'])
    context.bot.send_message(chat_id=update.effective_chat.id, text=str(df))

def last30(update: Update, context: CallbackContext):
    q=' '.join(context.args)
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,remaining_lease,storey_range&sort=month%20desc&limit=30&q="+q
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','remaining_lease','storey_range','floor_area_sqm','flat_model','month'])
    context.bot.send_message(chat_id=update.effective_chat.id, text=str(df))
    

In [3]:
    ### for testing API and response only
    url = "https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&fields=month,block,street_name,flat_type,flat_model,resale_price,floor_area_sqm,remaining_lease,storey_range&sort=month%20desc&limit=20&q="
    response=requests.get(url)
    dataset=response.json()
    df = pd.json_normalize(dataset['result'], "records")  
    df = df.sort_values(by='block').set_index('block').reindex(columns=['street_name','flat_type','resale_price','remaining_lease','storey_range','floor_area_sqm','flat_model','month'])
    print(df)


             street_name flat_type resale_price     remaining_lease  \
block                                                                 
117     ANG MO KIO AVE 4    3 ROOM       340000  55 years 02 months   
126     ANG MO KIO AVE 3    3 ROOM       380000  55 years 06 months   
128     ANG MO KIO AVE 3    3 ROOM       394888  55 years 06 months   
129     ANG MO KIO AVE 3    3 ROOM       350000  55 years 10 months   
154     ANG MO KIO AVE 5    3 ROOM       360000  57 years 10 months   
348     ANG MO KIO AVE 3    3 ROOM       375000   55 years 01 month   
404    ANG MO KIO AVE 10    3 ROOM       400000  56 years 02 months   
405    ANG MO KIO AVE 10    3 ROOM       345000  56 years 02 months   
418    ANG MO KIO AVE 10    3 ROOM       340000  56 years 05 months   
435    ANG MO KIO AVE 10    3 ROOM       335000  55 years 10 months   
435    ANG MO KIO AVE 10    3 ROOM       320000  55 years 10 months   
473    ANG MO KIO AVE 10    3 ROOM       338000  61 years 04 months   
533   

In [6]:
###
from telegram.ext import CommandHandler

start_handler = CommandHandler('start', start)
dispatcher.add_handler(start_handler)

last10_handler = CommandHandler('last10', last10)
dispatcher.add_handler(last10_handler)

last20_handler = CommandHandler('last20', last20)
dispatcher.add_handler(last20_handler)

last30_handler = CommandHandler('last30', last30)
dispatcher.add_handler(last30_handler)

bot.send_message(text='Launched!', chat_id=1349303768)

#####
updater.start_polling()



2022-04-15 15:07:44,682 - telegram.ext.updater - ERROR - Error while getting Updates: Conflict: terminated by other getUpdates request; make sure that only one bot instance is running
2022-04-15 15:07:44,683 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/telegram/ext/updater.py", line 646, in _network_loop_retry
    if not action_cb():
  File "/opt/conda/lib/python3.9/site-packages/telegram/ext/updater.py", line 597, in polling_action_cb
    updates = self.bot.get_updates(
  File "/opt/conda/lib/python3.9/site-packages/telegram/ext/extbot.py", line 224, in get_updates
    updates = super().get_updates(
  File "/opt/conda/lib/python3.9/site-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
  File "/opt/conda/lib/python3.9/site-packages/telegram/bot.py", line 3054, in get_updates
    self._post(
  File "/opt/conda/lib/python3.9/sit